In [2]:
# Loading in functions
from non_max_surpression import NonMaxSurpression
from model_dog_detection import DogDetectorModel
from image_to_tensor import to_tensor

# Loading libraries
import torch 
import os
import math
import pandas as pd

## Model Loading

In [2]:
# Initializing the model
model = DogDetectorModel()

In [3]:
# Initializing Non Max Surpression with default parameters
nms = NonMaxSurpression()

In [4]:
# Setting the device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CPU_DEVICE = torch.device("cpu")

In [5]:
# Loading the checkpoint
checkpoint = torch.load('dog-detection-model.pt')

# Loading the model weights
model.load_state_dict(checkpoint['model_state_dict'])

# Setting model to evaluation model
model = model.eval()

# Moving to training device
model = model.to(DEVICE)

## Dog Loading

In [3]:
dogInfo = pd.read_csv('attributes.csv', index_col=0)

In [ ]:
# dogInfo = dogInfo[dogInfo['path'] == 'dog_20']

In [9]:
dogInfo = dogInfo[dogInfo['downloadedImages'] > 0]

## Dog Cleaning

In [10]:
def split(a, n):
    # CITATION: https://stackoverflow.com/questions/2130016/splitting-a-list-into-n-parts-of-approximately-equal-length
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [11]:
numCleanedImagesAll = []
boxes = []

# Iterating through each dog
for index, dog in dogInfo.iterrows():
    if index % 65 == 0:
        print(f"{round((index / 6500) * 100, 3)} %")
        
    # Defining batch size of 2
    numBatches = math.ceil(dog['downloadedImages'] / 2)
    
    batchRanges = list(split(range(dog['downloadedImages']),numBatches))
    
    # Defining counter of the number of cleaned images
    numCleanedImages = 0
    
    for i in range(0,len(batchRanges)):
        batchRange = batchRanges[i]
    
        # Defining list to hold path to i
        imagePaths = []

        # Appending the path to each image
        for k in batchRange:
            imagePaths.append(f"dogs/{dog['path']}/img_{k}.jpg")

        # Loading and transforming images
        images = to_tensor(imagePaths)

        # Moving images to DEVICE
        images = list(image.to(DEVICE) for image in images)

        # Running image through model
        outputs = model(images, '')

        # Moving images to CPU
        images = list(image.to(CPU_DEVICE) for image in images)

        # Moving images back to CPU
        outputs = [{k: v.to(CPU_DEVICE) for k, v in t.items()} for t in outputs]

        # Executing non max surpression on the model output
        outputs = nms(outputs)
        
        # Iterating through the results and removing the bad pictures
        for j in range(0, len(imagePaths)):
            # If the image is not clean (i.e. no dog or more than 1 dog, we delete it)
            if len(outputs[j]['labels']) != 1:
                'bfjhdks'
                os.remove(f"dogs/{dog['path']}/img_{i*2 + j}.jpg")

            # If the image is clean, we just rename it to keep it inline with numCleanedImagesAll
            else:
                # Renaming the file
                os.rename(f"dogs/{dog['path']}/img_{i*2 + j}.jpg", f"dogs/{dog['path']}/img_{numCleanedImages}.jpg")

                # Extracting the bounding box
                box = outputs[j]['boxes'][0]

                # Recording the bounding box
                boxes.append(
                    {
                        'path':dog['path'], 
                        'image':f"img_{numCleanedImages}.jpg",
                        'xmin': box[0],
                        'ymin': box[1],
                        'xmax': box[2],
                        'ymax': box[3]

                    }
                )
                # Incrementing the number of cleaned images
                numCleanedImages += 1
            
            
    # Appending the cleaned image counts to the list
    numCleanedImagesAll.append(numCleanedImages)
    
    
# Appending the cleaned image counts to the df
dogInfo['cleanImages'] = numCleanedImagesAll

0.0 %


/home/aidan/Programs/miniconda3/envs/dogapp/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


1.0 %
2.0 %
3.0 %
4.0 %
5.0 %
6.0 %
7.0 %
8.0 %
9.0 %
10.0 %
11.0 %
12.0 %
13.0 %
14.0 %
15.0 %
16.0 %
17.0 %
18.0 %
19.0 %
20.0 %
21.0 %
22.0 %
23.0 %
24.0 %
25.0 %
26.0 %
27.0 %
28.0 %
29.0 %
30.0 %
31.0 %
32.0 %
33.0 %
34.0 %
35.0 %
36.0 %
37.0 %
38.0 %
39.0 %
40.0 %
41.0 %
42.0 %
43.0 %
44.0 %
45.0 %
46.0 %
47.0 %
48.0 %
49.0 %
50.0 %
51.0 %
52.0 %
53.0 %
54.0 %
55.0 %
56.0 %
57.0 %
58.0 %
59.0 %
60.0 %
61.0 %
62.0 %
63.0 %
64.0 %
65.0 %
66.0 %
67.0 %
68.0 %
69.0 %
70.0 %
71.0 %
72.0 %
73.0 %
74.0 %
75.0 %
76.0 %
77.0 %
78.0 %
79.0 %
80.0 %
81.0 %
82.0 %
83.0 %
84.0 %
85.0 %
86.0 %
87.0 %
88.0 %
89.0 %
90.0 %
91.0 %
92.0 %
93.0 %
94.0 %
95.0 %
96.0 %
97.0 %
98.0 %
99.0 %
100.0 %
101.0 %
102.0 %
103.0 %
104.0 %
105.0 %
106.0 %
107.0 %
108.0 %
109.0 %
110.0 %
111.0 %
112.0 %
113.0 %
114.0 %
115.0 %
116.0 %
117.0 %
118.0 %
119.0 %
120.0 %
121.0 %
122.0 %
123.0 %
124.0 %
125.0 %
126.0 %
127.0 %
128.0 %
129.0 %
130.0 %
131.0 %
132.0 %
133.0 %
134.0 %
135.0 %
136.0 %
137.0 %
138.0 %
139.

/tmp/ipykernel_13032/3295495394.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dogInfo['cleanImages'] = numCleanedImagesAll


In [15]:
# boxesDF = pd.DataFrame(boxes)
boxesDF.to_csv('boxesDF.csv')

In [16]:
dogInfo[dogInfo['cleanImages'] >= 2].reset_index(drop=True).to_csv('attributes.csv')

In [14]:
dogInfo[dogInfo['cleanImages'] >= 2]

name                                   breed     age     sex    size  \
0        mack  labrador retriever & border collie mix   young    male  medium   
1     twinkle                 german shepherd dog mix   young  female  medium   
2       bruce                    pit bull terrier mix   adult    male  medium   
3      caelum                  labrador retriever mix   adult    male   large   
4         sol                  labrador retriever mix   adult    male   large   
...       ...                                     ...     ...     ...     ...   
9723    bella                        pit bull terrier  senior  female  medium   
9724  butters                    pit bull terrier mix  senior    male  medium   
9725    betsy                    american bulldog mix  senior  female   large   
9726    kylee                    pit bull terrier mix  senior  female  medium   
9728    ollie          american staffordshire terrier  senior    male  medium   

           path  numImages  downloadedImages  \
0         dog_0          3                 3   
1         dog_1          6                 6   
2         dog_2          6                 6   
3         dog_4          4                 4   
4         dog_5          4                 4   
...         ...        ...               ...   
9723  dog_14993          4                 4   
9724  dog_14994          4                 4   
9725  dog_14995          3                 3   
9726  dog_14996          3                 3   
9728  dog_14999          3                 3   

                                                   link  cleanImages  
0     https://www.petfinder.com/dog/mack-54773637/bc...            3  
1     https://www.petfinder.com/dog/twinkle-54731090...            5  
2     https://www.petfinder.com/dog/bruce-54746372/b...            4  
3     https://www.petfinder.com/dog/caelum-52421265/...            2  
4     https://www.petfinder.com/dog/sol-52421267/bc/...            3  
...                                                 ...          ...  
9723  https://www.petfinder.com/dog/bella-50201066/c...            4  
9724  https://www.petfinder.com/dog/butters-49465364...            3  
9725  https://www.petfinder.com/dog/betsy-49465351/c...            2  
9726  https://www.petfinder.com/dog/kylee-49465353/c...            3  
9728  https://www.petfinder.com/dog/ollie-49465355/c...            3  

[8349 rows x 10 columns]

In [ ]:
# YOU DON"T NEED THIS PART ITS JUST FOR FUN

from plot_tensor import plot_tensor
import numpy as np

# Plotting image just for fun (you don't need this)
for i in range(0, len(images)):
    plot_tensor(images[i],np.array(outputs[i]['boxes']))